# Install Library

In [ ]:
!pip install wandb
# ----- Install ----- #

# ------------------- #
!wandb login

# Unzip files

In [ ]:
# ----- Unzip Files ----- #
# !tar -xvf '/content/drive/MyDrive/paper_implementation/Wavenet/datasets/LJSpeech-1.1.tar.bz2'

# ------------------- #

In [ ]:
# Load Library

In [ ]:
wandb_key = 'YOUR WANDB KEY'

# torch
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler



# linear algebra
import numpy as np

# vis
from matplotlib import pyplot as plt
import seaborn as sns

# util
import os
from glob import glob
import random
from easydict import EasyDict as edict
import re
import time
import wandb
from datetime import datetime
import yaml
import pandas as pd


from dataset import *
from utils import *
from models import *
from modules import *


import warnings
warnings.filterwarnings('ignore')



# ---------------  Set Base Directory ----------------- #
BASE_DIR = 'YOUR BASE DIRECTORY'
os.chdir(BASE_DIR)
os.getcwd()

# Fit

In [ ]:
with open(f"{BASE_DIR}/hparam.yaml") as f:
    hparams = edict(yaml.load(f))


hparams.save_path = '/content/drive/MyDrive/paper_implementation/Wavenet', # model save path


dataset_train = MyDataset()
dataset_val = MyDataset()

loader_train = DataLoader(dataset_train)
loader_val = DataLoader(dataset_val)

model = MyModel(hparams).cuda()

fitter = Fitter(model, hparams)
fitter.fit(loader_train, loader_val)